# Environment Setup

In [1]:
! pip install predibase
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.2 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tritonclient to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━

In [2]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig
from datasets import Dataset, load_dataset

pb = Predibase(api_token = 'INPUT_PREDIBASE_API_KEY_HERE')

Connected to Predibase as User(id=1a4e754d-8afe-46ac-bdb6-9cf2bf973088, username=loupdaniel12@gmail.com)

# Solar LLM (solar-1-mini-chat-240612) Fine-tuning

In [5]:
! pip install pandas

In [7]:
sample_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}

### Context:
{}

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""


# Function for formatting above prompt with information from Financial QA dataset
def formatting_prompts_func(examples):
    questions = examples["question"]
    contexts       = examples["context"]
    responses      = examples["answer"]
    texts = []
    for question, context, response in zip(questions, contexts, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = sample_prompt.format(question, context, response)
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("virattt/financial-qa-10K", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Function to rename columns
def rename_columns(example):
    example['prompt'] = example.pop('question')
    example['completion'] = example.pop('answer')
    return example

# Apply the renaming
dataset = dataset.map(rename_columns)

# Upload a dataset
dataset = pb.datasets.from_pandas_dataframe(dataset, name="financial-qa-10K")

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

In [8]:
# Create an adapter repository
repo = pb.repos.create(name="sec-10-k-chatbot", description="solar llm finetuning for creating a sec-10-k-chatbot", exists_ok=True)

In [9]:
# Create an adapter
adapter = pb.adapters.create(
   config=FinetuningConfig(
       base_model="solar-1-mini-chat-240612",
       epochs=10,
       rank=16,
       learning_rate=0.0002,
   ),
   dataset=dataset,
   repo=repo,
   description="solar llm finetuning for creating a sec-10-k-chatbot"
)

Successfully requested finetuning of solar-1-mini-chat-240612 as `sec-10-k-chatbot/1`. (Job UUID: b1e64fa1-e42a-4af7-850d-b67267195956).

Watching progress of finetuning job b1e64fa1-e42a-4af7-850d-b67267195956. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:05:47         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   1.0840   │        --       │
│     2      │   1.2849   │        --       │
│     3      │   1.0287   │        --       │
│     4      │   1.1419   │        --       │
│     5      │   0.9008   │        --       │
│     6      │   0.8630   │        --       │
└────────────┴────────────┴─────────────────┘


In [10]:
# Get adapter info after training
pb.adapters.get("sec-10-k-chatbot/1")

Adapter(repo='sec-10-k-chatbot', tag=1, archived=False, base_model='solar-1-mini-chat-240612', description='solar llm finetuning for creating a sec-10-k-chatbot', artifact_path='b1e64fa1-e42a-4af7-850d-b67267195956/60940739f42a41ebb5a9a5df17f287ef/artifacts/model/model_weights', finetuning_error=None, finetuning_job_uuid='b1e64fa1-e42a-4af7-850d-b67267195956')

In [11]:
adapter

Adapter(repo='sec-10-k-chatbot', tag=1, archived=False, base_model='solar-1-mini-chat-240612', description='solar llm finetuning for creating a sec-10-k-chatbot', artifact_path='b1e64fa1-e42a-4af7-850d-b67267195956/60940739f42a41ebb5a9a5df17f287ef/artifacts/model/model_weights', finetuning_error=None, finetuning_job_uuid='b1e64fa1-e42a-4af7-850d-b67267195956')

In [12]:
adapter_id = adapter.repo + "/" + str(adapter.tag)
adapter_id

'sec-10-k-chatbot/1'

# Run Inference on the Fine-tuned Adapter

In [13]:
text = "What were the primary drivers of the notable increase in research and development expenses from Apple for fiscal year 2023?"
# Prompt the adapter to generate response via serverless endpoints
print(lorax_client.generate(text, adapter_id="sec-10-k-chatbot/1").generated_text)

The primary drivers of the 28% increase in research and development expenses for fiscal year 2023 were higher employee-related costs due to increased headcount and higher performance-based bonuses, as well as an increase in engineering project costs and data center infrastructure.


# Appendix: Data Preprocessing

In [ ]:
import pandas as pd

# Import the original dataset
df = pd.read_csv("customer_complaint_demo_data_raw.csv")
df = df.rename(columns={"Consumer complaint narrative": "Complaint", "Structured JSON Output": "completion"})
df.head()

In [ ]:
tpl = """You are a support agent for a public financial company and a customer has raised a complaint.
    Generate a structured JSON output with the following fields "product", "issue", and "generatedCompanyResponse".

    Here is an example structure:
    {{
      "product": "...",
      "issue": "...",
      "generatedCompanyResponse": "..."
    }}

    The value for "generatedCompanyResponse" should be a polite response to the following complaint.

    ### Complaint: {Complaint}

    ### Structured JSON Output:"""

output_column_name = "completion"

In [ ]:
# process the data to fit in the prompt teamplate
new_df = pd.DataFrame()
prompts = []
for index, row in df.iterrows():
    prompt = tpl.format(**row.to_dict())
    prompts.append(prompt)
new_df["prompt"] = prompts
new_df["completion"] = df[output_column_name]

print(new_df.head())

                                              prompt  \
0  You are a support agent for a public financial...   
1  You are a support agent for a public financial...   
2  You are a support agent for a public financial...   
3  You are a support agent for a public financial...   
4  You are a support agent for a public financial...   

                                          completion  
0  {\n"product": "Student loan",\n"issue": "Strug...  
1  {\n"product": "Credit reporting",\n"issue": "I...  
2  {\n"product": "Credit card or prepaid card",\n...  
3  {\n"product": "Credit card",\n"issue": "Payoff...  
4  {\n"product": "Money transfer, virtual currenc...  
